# Scale your pandas workflows by changing one line of code #
- https://github.com/modin-project/modin
    
## With Pandas ##
```
import pandas as pd
```
## With modin pandas ##
```
import modin.pandas as pd
```

In [ ]:
!python --version

In [ ]:
!pip show pandas

In [ ]:
!pip show modin

In [ ]:
!ls  -lha ../wildfire/wildfire-data 

In [ ]:
from datetime import datetime
import pandas as pdo 
import modin.pandas as pd

In [ ]:
from modin.config import Engine

In [ ]:
Engine.put("dask")

In [ ]:
from distributed import Client
client = Client()

In [ ]:
!ls -lah

In [ ]:
source = "mix_fire.csv"

## Reading CSV (not zipped)

In [ ]:
t0 = datetime.now()
df_pd = pdo.read_csv(source, low_memory=False)
end = datetime.now()
end.strftime('%Y-%m-%d %H:%M:%S')
f'Total time {(end - t0).seconds} (s)'

In [ ]:
df_pd.shape

In [ ]:
t0 = datetime.now()
df = pd.read_csv(source)
end = datetime.now()
end.strftime('%Y-%m-%d %H:%M:%S')
f'Total time {(end - t0).seconds} (s)'

In [ ]:
df.shape

## Reading gzip CSV (Zipped)

In [ ]:
source_gz = 'mix_fire.csv.gz'

In [ ]:
t0 = datetime.now()
df_pd_gz = pdo.read_csv(source_gz, low_memory=False)
end = datetime.now()
end.strftime('%Y-%m-%d %H:%M:%S')
f'Total time {(end - t0).seconds} (s)'

In [ ]:
t0 = datetime.now()
df_mo_gz = pd.read_csv(source_gz)
end = datetime.now()
end.strftime('%Y-%m-%d %H:%M:%S')
f'Total time {(end - t0).seconds} (s)'

## checking dataframes

In [ ]:
print(df_pd_gz.shape)
print(df_mo_gz.shape)

In [ ]:
PRECISION = 1

## Using Modin Pandas

In [ ]:
t_start = datetime.now()
## ----
df = df_pd_gz[['latitude', 'longitude', 'acq_date', 'confidence']]
df.latitude = df.latitude.round(PRECISION)
df.longitude = df.longitude.round(PRECISION)
df['acq_date'] = pd.to_datetime(df['acq_date'])
df['year'] = df.acq_date.dt.year
df['month'] = df.acq_date.dt.month
total_fires = df.groupby(['latitude', 'longitude', 'year', 'month']).size().reset_index()
total_fires.columns = ['latitude', 'longitude', 'year', 'month', 'fire_count']
yearly_fires = total_fires.groupby(['longitude', 'latitude', 'year', ]).count().reset_index()
global_fires = total_fires.groupby(['latitude', 'longitude']).sum().reset_index()
# ----
t_end = datetime.now()
t_end.strftime('%Y-%m-%d %H:%M:%S')
f'Total time {(t_end - t_start).seconds} (s)'

In [ ]:
print(yearly_fires.shape)
yearly_fires

In [ ]:
print(global_fires.shape)
global_fires

In [ ]:
import plotly.express as px
import plotly.figure_factory as ff
import numpy as np
MAPBOX_TOKEN = 'pk.eyJ1IjoiZm9kZ2Fib3JtYXRoIiwiYSI6ImNrZmY3Nzc2bjBiemkyeG8zdGNzcXgzMGIifQ.J0dZhMiuZTPVexL8nrpS6Q'
px.set_mapbox_access_token(MAPBOX_TOKEN)

In [ ]:
t_start = datetime.now()
## ----
fig = ff.create_hexbin_mapbox(
    data_frame=global_fires, lat='latitude', lon='longitude',
    nx_hexagon=100, opacity=0.9, labels={"color": "Hotspot records"},
    color='fire_count', agg_func=np.sum, color_continuous_scale="Reds"
)
fig.show()
# ----
t_end = datetime.now()
t_end.strftime('%Y-%m-%d %H:%M:%S')
f'Total time {(t_end - t_start).seconds} (s)'

## Using Pandas 

In [ ]:
t_start = datetime.now()
## ----
df = df_mo_gz[['latitude', 'longitude', 'acq_date', 'confidence']]
df.latitude = df.latitude.round(PRECISION)
df.longitude = df.longitude.round(PRECISION)
df['acq_date'] = pd.to_datetime(df['acq_date'])
df['year'] = df.acq_date.dt.year
df['month'] = df.acq_date.dt.month
total_fires = df.groupby(['latitude', 'longitude', 'year', 'month']).size().reset_index()
total_fires.columns = ['latitude', 'longitude', 'year', 'month', 'fire_count']
yearly_fires = total_fires.groupby(['longitude', 'latitude', 'year', ]).count().reset_index()
global_fires = total_fires.groupby(['latitude', 'longitude']).sum().reset_index()
# ----
t_end = datetime.now()
t_end.strftime('%Y-%m-%d %H:%M:%S')
f'Total time {(t_end - t_start).seconds} (s)'

In [ ]:
print(global_fires.shape)
global_fires

In [ ]:
t_start = datetime.now()
## ----
fig = ff.create_hexbin_mapbox(
    data_frame=global_fires, lat=0, lon=1,
    nx_hexagon=100, opacity=0.9, labels={"color": "Hotspot records"},
    color=4, agg_func=np.sum, color_continuous_scale="Reds"
)
fig.show()
# ----
t_end = datetime.now()
t_end.strftime('%Y-%m-%d %H:%M:%S')
f'Total time {(t_end - t_start).seconds} (s)'

In [ ]:
global_fires.columns

In [ ]:
fig = px.density_mapbox(global_fires, lat=0, lon=1, z=4, radius=10,
                        center=dict(lat=0, lon=180), zoom=0,
                        mapbox_style="stamen-terrain")
fig.show()